In [1]:
import what.utils.logger as log

logger = log.get_logger(__name__)

In [2]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
pretrained_ssd = "models/mobilenet-v2-ssd-lite-mp-0_686.pth"
checkpoint_folder = "models/"

# Load Training Dataset

In [4]:
batch_size = 32
num_workers = 0

In [5]:
import fiftyone as fo
import fiftyone.zoo as foz
from torch.utils.data import DataLoader

from what.models.detection.datasets.fiftyone import FiftyOneDataset

In [6]:
from what.models.detection.ssd.ssd.preprocessing import TrainAugmentation, TestTransform
from what.models.detection.ssd.ssd import mobilenet_ssd_config
from what.models.detection.ssd.ssd.ssd import MatchPrior

In [7]:
# Load MobileNetSSD configuration
config              = mobilenet_ssd_config
train_transform     = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
target_transform    = MatchPrior(config.priors, config.center_variance, config.size_variance, 0.5)
test_transform      = TestTransform(config.image_size, config.image_mean, config.image_std)

In [8]:
# Load the data from fiftyone
voc_2012_train = foz.load_zoo_dataset("voc-2012", split="train")

Split 'train' already downloaded
Loading 'voc-2012' split 'train'
 100% |███████████████| 5717/5717 [8.9s elapsed, 0s remaining, 581.7 samples/s]      
Dataset 'voc-2012-train' created


### Visualize the dataset

In [ ]:
session = fo.launch_app(voc_2012_train)

In [9]:
# Convert to pytorch dataloader
train_dataset = FiftyOneDataset(voc_2012_train, 
                                foz.load_zoo_dataset_info("voc-2012").classes, 
                                transform=train_transform,
                                target_transform=target_transform)

# Load Training Dataset from Local Disk
# train_dataset = VOCDataset("examples/VOC2012", transform=train_transform,
#                              target_transform=target_transform)

train_loader  = DataLoader(train_dataset, batch_size,
                          num_workers=num_workers,
                          shuffle=True)

logger.info("Train dataset size: {}".format(len(train_dataset)))

[INFO] 2022-06-20 19:55:11,456 __main__: Train dataset size: 5717


# Load Validation Dataset

In [10]:
# Load Validation Dataset from FiftyOne (use voc-2007 train as validation here)
val_dataset  = FiftyOneDataset(foz.load_zoo_dataset("voc-2007", split="train"), 
                                        foz.load_zoo_dataset_info("voc-2007").classes,
                                        transform=test_transform,
                                        target_transform=target_transform)

# Load Validation Dataset from Local Disk
# val_dataset = VOCDataset("VOC2007/", transform=test_transform,
#                              target_transform=target_transform, is_test=True)

val_loader = DataLoader(val_dataset, batch_size,
                        num_workers=num_workers,
                        shuffle=False)

logger.info("validation dataset size: {}".format(len(val_dataset)))

Split 'train' already downloaded
Loading 'voc-2007' split 'train'
 100% |███████████████| 2501/2501 [4.0s elapsed, 0s remaining, 590.9 samples/s]      
Dataset 'voc-2007-train' created


[INFO] 2022-06-20 19:55:16,834 __main__: validation dataset size: 2501


# Train Mobilenet V2 SSD Lite

In [11]:
from what.models.detection.ssd.mobilenet_v2_ssd_lite import MobileNetV2SSDLite

In [12]:
# Create SSD network and load pretrained base net.
model = MobileNetV2SSDLite(is_test=False, class_names=train_dataset.classes)

model.train(train_loader, val_loader, device=device, num_epochs=1, debug_steps=10, validation_epochs=1,
           pretrained_ssd = pretrained_ssd, checkpoint_folder = checkpoint_folder)

[INFO] 2022-06-20 19:55:16,996 what.models.detection.ssd.mobilenet_v2_ssd_lite: Init from pretrained ssd models/mobilenet-v2-ssd-lite-mp-0_686.pth
[INFO] 2022-06-20 19:55:18,670 what.models.detection.ssd.mobilenet_v2_ssd_lite: Start training using CosineAnnealingLR scheduler.
/home/wuhanstudio/Desktop/whitebox-adversarial-toolbox/what/models/detection/ssd/transforms/transforms.py:244: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
[INFO] 2022-06-20 19:55:23,569 what.models.detection.ssd.mobilenet_v2_ssd_lite: Epoch: 0, Step: 10, Average Loss: 22.4182, Average Regression Loss 4.2713, Average Classification Loss: 18.1469
[INFO] 2022-06-20 19:55:27,751 what.models.detection.ssd.mobilenet_v2_ssd_lite: Epoch: 0, Step: 20, Average Los